# Churn Server


In [ ]:
import numpy as np
from cloudpickle import load


import mlrun
class ChurnModel(mlrun.serving.V2ModelServer):
    def load(self):
        """
        load multiple models in nested folders, churn model only
        """
        clf_model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(str(clf_model_file), "rb"))
        if "cox" in extra_data.keys():
            cox_model_file = extra_data["cox"]
            self.cox_model = load(open(str(cox_model_file), "rb"))
            if "cox/km" in extra_data.keys():
                km_model_file = extra_data["cox/km"]
                self.km_model = load(open(str(km_model_file), "rb"))

    def predict(self, body):
        try:
            # we have potentially 3 models to work with:
            #if hasattr(self, "cox_model") and hasattr(self, "km_model"):
                # hack for now, just predict using one:
            feats = np.asarray(body["instances"], dtype=np.float32).reshape(-1, 23)
            result = self.model.predict(feats, validate_features=False)
            return result.tolist()
            #else:
            #    raise Exception("models not found")
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [ ]:
# nuclio: end-code

## Configuration


In [ ]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or "http://mlrun-api:8080"
mlconf.artifact_path = mlconf.artifact_path or f"{os.environ['HOME']}/artifacts"


In [ ]:
import pandas as pd

DATA_URL = f"https://raw.githubusercontent.com/yjb-ds/testdata/master/demos/churn/churn-tests.csv"
xtest = pd.read_csv(DATA_URL)


<a id="deploy"></a>
### Deploy our serving class using as a serverless function
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the assosiated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

**verify the model dir does contain a valid `model.bst` file**

In [ ]:
from mlrun import import_function
from mlrun.platforms.other import auto_mount
import requests

In [ ]:
fn = import_function("hub://churn_server")

model_dir = os.path.join(mlconf.artifact_path, "churn", "models")
fn.add_model("churn_server_v1", model_path=model_dir)

fn.apply(auto_mount())

## Tests

In [ ]:
addr = fn.deploy()

### **test our model server using HTTP request**
We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [ ]:
# KFServing protocol event
event_data = {"instances": xtest.values[:10,:-1].tolist()}

In [ ]:
import json
resp = requests.put(addr + "/churn_server_v1/predict", json=json.dumps(event_data))

tl = resp.text.replace("[","").replace("]","").split(",")
tl

**[back to top](#top)**